##### Example use of the hyperopt to see the sensitivy of an algorith with respect ot their hyperparameters
Here the use-case is a RandomForest algo and see the importance of different hyperaparameters based on the accuracy on the train and test set.
Simple train/test split has been performed.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier



space = {
    'criterion': hp.choice( 'criterion', ( 'gini', 'entropy' )),
    'bootstrap': hp.choice( 'bootstrap', ( True, False )),
    'class_weight': hp.choice( 'class_weight', ( 'balanced', 'balanced_subsample', None )),
    'max_depth': hp.choice('max_depth', np.arange( 2, 50, dtype=int)),
    'max_features': hp.choice( 'mf', ( 'sqrt', 'log2', None )),
    'min_samples_split': hp.choice('min_samples_split', np.arange( 2, 20, dtype=int)),
    'min_samples_leaf': hp.choice('min_samples_leaf', np.arange( 1, 20, dtype=int)),
    'n_estimators': hp.choice( 'n_estimators', [100,150, 200, 250, 300, 350,500]),
}




random_state = 42

X_train, X_cv, y_train, y_cv = train_test_split(X_B,
                                                y, 
                                                test_size=0.1, 
                                                random_state=random_state, 
                                                stratify=y)


def objective(space):

    clf = RandomForestClassifier(random_state=random_state, **space)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_cv)
    acc = accuracy_score(y_cv, pred)
    train_acc = accuracy_score(y_train, clf.predict(X_train))
    print "Train score: %0.3f  -- Validation score: %0.3f"% (train_acc, acc)
    to_return = {'loss':1-acc, 'train_loss':1-train_acc, 'status': STATUS_OK }
    to_return.update(space)
    return to_return


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print best


NameError: name 'X_B' is not defined

# Fetch the trials results
And keep a separate Dataframe for the categorical variables

In [2]:
tt = [t['result'] for t in trials.trials]
dd = pd.DataFrame(tt)
categorical_feats = ['criterion', 
                     'class_weight', 
                     'max_features',
                     'bootstrap']
dd_cat = dd[categorical_feats+['loss']+['train_loss']]
for cat in categorical_feats:
    del dd[cat]
del dd['status']
print dd.shape
print dd.columns

NameError: name 'trials' is not defined

# Pivot the table
(this can be done more efficiently probably)

In [5]:
tr = []
for i, row in dd.iterrows():
    cur = {}
    cur['loss'] = row['loss']
    train_loss = row['train_loss']
    del row['loss']
    del row['train_loss']
    cur['set'] = "Test"
    cur.update(row.to_dict())
    tr.append(cur)
    cur = {}
    cur['loss'] = train_loss
    cur['set'] = "Train"
    cur.update(row.to_dict())
    tr.append(cur)
dd2 = pd.DataFrame(tr)
dd2

NameError: name 'dd' is not defined

# Do the same for the categorical Dataframe


In [7]:
tr = []
for i, row in dd_cat.iterrows():
    cur = {}
    cur['loss'] = row['loss']
    train_loss = row['train_loss']
    del row['loss']
    del row['train_loss']
    cur['set'] = "Test"
    cur.update(row.to_dict())
    tr.append(cur)
    cur = {}
    cur['loss'] = train_loss
    cur['set'] = "Train"
    cur.update(row.to_dict())
    tr.append(cur)
dd2_cat = pd.DataFrame(tr)
dd2_cat

NameError: name 'dd_cat' is not defined

### Plot the Results

In [ ]:
dd2_melted = dd2.melt(['loss', 'set'])
g = sns.lmplot(x="value", y="loss", col="variable", hue='set', data=dd2_melted,sharex=False) 

### Plot the Results
(Categorical Vars into boxplot)
This needs some fixing for the empty labels

In [ ]:
dd2_cat_melted = dd2_cat.melt(['loss', 'set'])
g = sns.catplot(x="value", y="loss", col='variable', hue='set', 
                data=dd2_cat_melted, sharex=False,
                kind='box')